# **SQL Agent**
"AI agents are software systems that use AI to pursue goals and complete tasks on behalf of users. They show reasoning, planning, and memory and have a level of autonomy to make decisions, learn, and adapt." ~ **Google**

"An artificial intelligence (AI) agent is a system that autonomously performs tasks by designing workflows with available tools." ~ **IBM**

"An artificial intelligence (AI) agent is a software program that can interact with its environment, collect data, and use that data to perform self-directed tasks that meet predetermined goals. Humans set goals, but an AI agent independently chooses the best actions it needs to perform to achieve those goals." ~ **Amazon Web Services (AWS)**

<br>

**AI agents** are software systems that autonomously perform tasks by interacting with their environments (or external environments) through ***reasoning***, ***planning***, ***perception***, and ***proactivity***. They can obtain data from external sources (or systems) and tools via APIs, websites, human-in-the-loop (HITL) processes, and interacting with other AI agents.

AI agents solve complicated and sophisticated tasks across enterprise applications, including software design, IT automation, code generation and conversational assistance. They apply advanced natural language processing techniques of large language models (LLMs) to comprehend and respond to user inputs step-by-step and determine when to call on external tools.

<br>

---


In this notebook, I experimented with and developed an artificial intelligence (ai) agent that can provide answers about a SQL database. This agent or system takes a user's question as its input and converts the input (questions) into queries that can be parsed, optimized, and executed by a SQL (or database) engine.

<br>

Broadly, the following steps listed below describe what the agent will do:
1. Fetch the schemas and available tables from the database.
2. Decide which tables are pertinent to the question.
3. Fetch the schemas for the relevant tables.
4. Generate a sql query based on the question and information from the schemas.
5. Double-check the sql query for common mistakes using a LLM.
6. Execute the query and return the results.
7. Correct mistakes surfaced by the database engine until the query is successful.
8. Formulate a response based on the results.

**Install the necessary libraries for the project.**

In [1]:
!pip install --quiet --upgrade langchain  langgraph  langchain-community

In [2]:
!pip install -qU "langchain[google-genai]"

**Import all the necessary libraries, modules or packages**

In [17]:
import os
import getpass
import pathlib
import requests
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_community.utilities import SQLDatabase
from langchain_community.agent_toolkits import SQLDatabaseToolkit
from langchain.agents import create_agent
from langchain.agents.middleware import HumanInTheLoopMiddleware
from langgraph.checkpoint.memory import InMemorySaver
from langgraph.types import Command

**Check libraies, modules or packages version**

In [24]:
import sys
import requests
import langchain
import langchain_community
import langgraph

print("Python:", sys.version)
print("requests:", requests.__version__)
print("langchain:", langchain.__version__)
print("langchain_community:", langchain_community.__version__, "\n")
!pip show langgraph

Python: 3.12.12 (main, Oct 10 2025, 08:52:57) [GCC 11.4.0]
requests: 2.32.5
langchain: 1.2.0
langchain_community: 0.4.1 

Name: langgraph
Version: 1.0.5
Summary: Building stateful, multi-actor applications with LLMs
Home-page: https://docs.langchain.com/oss/python/langgraph/overview
Author: 
Author-email: 
License: 
Location: /usr/local/lib/python3.12/dist-packages
Requires: langchain-core, langgraph-checkpoint, langgraph-prebuilt, langgraph-sdk, pydantic, xxhash
Required-by: langchain


In [4]:
def _set_env(key: str):
    """
    ensure that an environment variable is set.

    if the environment variable specified by 'key' does not exist,
    prompt the user to input the value securely (without echoing)
    and store it in the environment. If the variable already exists,
    simply notify the user.

    Args:
        key (str): The name of the environment variable to check/set.
    """
    if key not in os.environ:
        os.environ[key] = getpass.getpass(f"Enter your {key}: ")
        print(f"{key} captured.")
    else:
        print(f"{key} is in environment")


# calls for google and langsmith api keys.
_set_env("GOOGLE_API_KEY")
_set_env("LANGSMITH_API_KEY")

Enter your GOOGLE_API_KEY: ··········
GOOGLE_API_KEY captured.
Enter your LANGSMITH_API_KEY: ··········
LANGSMITH_API_KEY captured.


#### Initializing the Large Language Model (LLM)

In this step, we initialize a Google Gemini chat-based LLM using LangChain's `ChatGoogleGenerativeAI` wrapper. The selected model (`gemini-2.5-flash-lite`) is fastest flash model optimized for cost-efficiency and high throughput, making it suitable for conversational agents and interactive workflows.

In [18]:
# initialize a google gemini chat model to be used as the LLM.
model = ChatGoogleGenerativeAI(model="gemini-3-flash-preview")

# model = ChatGoogleGenerativeAI(model="gemini-3-pro-preview")

#### **Setup the SQLite database.**
The sample database used in this implementation is the ***chinook database***, which is a popular, free sample database modeling a digital media store.

It contains 11 tables, featuring tables for Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, and Track. It is available in various SQL formats like SQLite, MySQL, PostgreSQL, SQL Server, and Oracle.


In [21]:
# url of the Chinook SQLite database to download.
url = "https://storage.googleapis.com/benchmarks-artifacts/chinook/Chinook.db"

# local path where the database will be saved.
local_path = pathlib.Path("Chinook.db")

# check if the file already exists locally.
if local_path.exists():
    print("chinook database already exists! skip downloading.")

else:
    # download the file from the url.
    response = requests.get(url)

    # if the download is successful, save the binary file locally.
    if response.status_code == 200:
        local_path.write_bytes(response.content)
        print(f"file downloaded and saved as {local_path}")

    else:
        # print an error message if the download failed.
        print(f"failed to download the file. Status code: {response.status_code}")

chinook database already exists! skip downloading.


#### **Connecting to the SQL Database and Inspecting Its Structure**

In this section, we establish a connection to a local SQLite database using LangChain's `SQLDatabase` utility.

After creating the database connection, we inspect key metadata about the database, including the SQL dialect in use and the list of available tables. Finally, we execute a sample SQL query to verify that the connection is working correctly and to preview the structure of the data.

In [8]:
# create a connection to the SQLite database using its uri.
db = SQLDatabase.from_uri("sqlite:///Chinook.db")

# print the SQL dialect used by the connected database.
print(f"dialect: {db.dialect}")

# retrieve and display the list of usable tables in the chinook database.
print(f"available tables: {db.get_usable_table_names()}")

# execute a sample SQL query to fetch a few rows from the Artist table, and print the output to verify database access.
print(f"sample Output: {db.run('SELECT * FROM Artist LIMIT 5;')}")

dialect: sqlite
available tables: ['Album', 'Artist', 'Customer', 'Employee', 'Genre', 'Invoice', 'InvoiceLine', 'MediaType', 'Playlist', 'PlaylistTrack', 'Track']
sample Output: [(1, 'AC/DC'), (2, 'Accept'), (3, 'Aerosmith'), (4, 'Alanis Morissette'), (5, 'Alice In Chains')]


#### **Inspecting SQL Agent Tools**

In this section, we create a SQL database toolkit by pairing a database connection with an LLM. The toolkit exposes a set of predefined tools that enable an agent to inspect database schemas, generate SQL queries, and execute them. Printing the tool names and descriptions helps verify and understand the capabilities available to the agent.

In [19]:
# initialize the SQL database toolkit by combining the connected database with the LLM that will reason about SQL queries.
toolkit = SQLDatabaseToolkit(db=db, llm=model)

# Extract the list of SQL-related tools provided by the toolkit. these tools define the actions the agent can perform on the database.
tools = toolkit.get_tools()

# iterate through each tool and display its name and description for inspection and understanding of the agent's available capabilities.
for tool in tools:
    print(f"{tool.name}: {tool.description}\n")

sql_db_query: Input to this tool is a detailed and correct SQL query, output is a result from the database. If the query is not correct, an error message will be returned. If an error is returned, rewrite the query, check the query, and try again. If you encounter an issue with Unknown column 'xxxx' in 'field list', use sql_db_schema to query the correct table fields.

sql_db_schema: Input to this tool is a comma-separated list of tables, output is the schema and sample rows for those tables. Be sure that the tables actually exist by calling sql_db_list_tables first! Example Input: table1, table2, table3

sql_db_list_tables: Input is an empty string, output is a comma-separated list of tables in the database.

sql_db_query_checker: Use this tool to double check if your query is correct before executing it. Always use this tool before executing a query with sql_db_query!



#### **Let's build the Reasoning and Action (`ReAct`) agent**.

The agent will interpret the request and generate a SQL command, which the tools will execute. If the command has an error, the error message is sent back to the model. The model can then scrutinize the original request and the new error message to generate a new command.

This process or flow can continue until the LLM generates the command successfully or reaches an end count. This pattern of providing a model with feedback - error messages in this case - not only makes it powerful but also robust.

In [20]:
# set the maximum number of results to return unless the user specifies otherwise.
top_k=5

# define the system prompt to customize the agent's behavior and instructs it on how to interact with the SQL database.
system_prompt = f"""
You are an agent designed to interact with a SQL database.
Given an input question, create a syntactically correct {db.dialect} query to run, then look at the results of the query and return the answer.
Unless the user specifies a specific number of examples they wish to obtain, always limit your query to at most {top_k} results.

You can order the results by a relevant column to return the most interesting examples in the database. Never query for all the columns from a specific table,
only ask for the relevant columns given the question.

You MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.

DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the
database.

To start you should ALWAYS look at the tables in the database to see what you
can query. Do NOT skip this step.

Then you should query the schema of the most relevant tables.

Once the query executes successfully, **use the results to generate a clear, concise, and conversational response** to the user.
Make sure your response reflects only the information obtained from the executed query.
"""

# create the ai agent with the specified model, tools, and system prompt.
agent = create_agent(
    model,
    tools,
    system_prompt=system_prompt
)

# define the user question to ask the agent.
question = "Which genre on average has the longest tracks?"

# stream the agent's responses step by step.
for step in agent.stream(
    {"messages": [{"role": "user", "content": question}]},
    stream_mode="values",
):

    # print the latest message from the agent in a readable format.
    step["messages"][-1].pretty_print()

================================ Human Message =================================

Which genre on average has the longest tracks?


ChatGoogleGenerativeAIError: Error calling model 'gemini-3-flash-preview' (RESOURCE_EXHAUSTED): 429 RESOURCE_EXHAUSTED. {'error': {'code': 429, 'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/usage?tab=rate-limit. \n* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 20, model: gemini-3-flash\nPlease retry in 37.313838921s.', 'status': 'RESOURCE_EXHAUSTED', 'details': [{'@type': 'type.googleapis.com/google.rpc.Help', 'links': [{'description': 'Learn more about Gemini API quotas', 'url': 'https://ai.google.dev/gemini-api/docs/rate-limits'}]}, {'@type': 'type.googleapis.com/google.rpc.QuotaFailure', 'violations': [{'quotaMetric': 'generativelanguage.googleapis.com/generate_content_free_tier_requests', 'quotaId': 'GenerateRequestsPerDayPerProject

#### **Implement the Human-In-The-Loop Review**

Due to inefficiencies, unintended or unexpected actions, it is advisable and highly recommended to check the ai agent's SQL queries before they are executed. The human-in-the-loop review process will help add human oversight to agent tool calls. When a model proposes an action that might require review - for example, writing to a file or executing SQL - the middleware can pause execution and wait for a decision from the human.

In [13]:
# create an ai agent with the specified model, tools, and system prompt.
agent = create_agent(
    model,
    tools,
    system_prompt=system_prompt,

    # add middleware for implementation of the human-in-the-loop (HITL) review.
    middleware=[
        HumanInTheLoopMiddleware(
            interrupt_on={"sql_db_query": True},
            description_prefix="Tool execution pending approval",
        ),
    ],

     # checkpointing to save the agent's state in memory.
    checkpointer=InMemorySaver(),
)

In [15]:
# define the user question to ask the agent.
question = "Which genre on average has the longest tracks?"

# define a configuration for the agent.
config = {"configurable": {"thread_id": "1"}}

# stream the agent's response step by step.
for step in agent.stream(
    {"messages": [{"role": "user", "content": question}]},
    config,
    stream_mode="values",
):

    # check if the agent's execution was interrupted.
    if "__interrupt__" in step:
        print("INTERRUPTED:")
        interrupt = step["__interrupt__"][0]

        # print the descriptions of all action requests that require human approval.
        for request in interrupt.value["action_requests"]:
            print(request["description"])

    # if there are normal messages from the agent (that is, no interrupt), print the last message in a readable format.
    elif "messages" in step:
        step["messages"][-1].pretty_print()
    else:
        pass

================================ Human Message =================================

Which genre on average has the longest tracks?


ChatGoogleGenerativeAIError: Error calling model 'gemini-3-flash-preview' (RESOURCE_EXHAUSTED): 429 RESOURCE_EXHAUSTED. {'error': {'code': 429, 'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/usage?tab=rate-limit. \n* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 20, model: gemini-3-flash\nPlease retry in 36.263974321s.', 'status': 'RESOURCE_EXHAUSTED', 'details': [{'@type': 'type.googleapis.com/google.rpc.Help', 'links': [{'description': 'Learn more about Gemini API quotas', 'url': 'https://ai.google.dev/gemini-api/docs/rate-limits'}]}, {'@type': 'type.googleapis.com/google.rpc.QuotaFailure', 'violations': [{'quotaMetric': 'generativelanguage.googleapis.com/generate_content_free_tier_requests', 'quotaId': 'GenerateRequestsPerDayPerProject

#### Using `Command` to resume the agent execution after interruption. Here, resuming is accepting the query if it is accurate.


In [ ]:
# stream the agent's response after resuming with an approval command.
for step in agent.stream(
    Command(resume={"decisions": [{"type": "approve"}]}),
    config,
    stream_mode="values",
):
    if "messages" in step:
        step["messages"][-1].pretty_print()
    elif "__interrupt__" in step:
        print("INTERRUPTED:")
        interrupt = step["__interrupt__"][0]
        for request in interrupt.value["action_requests"]:
            print(request["description"])
    else:
        pass